# `Промышленное машинное обучение на Spark`
## `Занятие 03: Основы Spark`

О чём можно узнать из этого ноутбука:

* DataFrame и SQL API
* Базовые операции в Spark

### `Устанавливаем необходимые зависимости`

In [4]:
! pip3 install -q pyspark pyarrow parquet-tools

### `Готовим SparkContext`

Как следует из лекции объект SparkContext является точкой входа для работы со Spark кластером. 

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

# Создаём конфигурационный класс с параметрами подключения
conf = (
    SparkConf()
        # Указываем порт на котором будет располагаться UI
        .set('spark.ui.port', '4050')
        # Указываем URL master ноды Spark кластера
        # Можно использовать local mode, указав `local[<number_cores>]`
        # В таком случае вся обработка будет происходить на текущем компьютере
        # При этом, это может давать преимущество ввиду наличия параллелизма по ядрам компьютера
        .setMaster('local[*]')
        # Если нужно подключиться к "реальному" кластеру то нужно указать URL `spark://<master-node-url:master-node-url>`. Например:
        # .setMaster('spark://localhost:7077')
)
# Создаём точку доступа на кластер. Позволят использовать RDD API
sc = SparkContext(conf=conf)
# Точка доступа для использования DataFrame API
spark = SparkSession(sc)

# По завершении программы нужно обязательно выполнить остановку подключения для освобождения занятых ресурсов
# sc.stop()

23/10/20 22:03:48 WARN Utils: Your hostname, vm-01 resolves to a loopback address: 127.0.1.1; using 10.128.0.16 instead (on interface eth0)
23/10/20 22:03:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 22:03:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


После запуска spark у нас доступен UI интерфейс, где будет отображаться информация о Spark-кластере, запущенных задачах и выводы различных логов.

Чтобы перейти в UI интерфейс достаточно в новом окне браузера ввести: <Адрес вашей виртуальной машины>:4050/

### `Загрузка данных`

Скачаем данные, с которыми будем в дальнейшем работать

In [29]:
import json
import requests
import urllib

folder_url = 'https://disk.yandex.lt/d/JnDy1h48pJI7IA'
file_url = '/m5-forecating-accuracy.zip'
# запрос ссылки на скачивание
response = requests.get('https://cloud-api.yandex.net/v1/disk/public/resources/download',
                 params={'public_key': folder_url, 'path': file_url}) 
# 'парсинг' ссылки на скачивание
data_link = response.json()['href'] 	
data_link

'https://downloader.disk.yandex.ru/disk/4ac806482fc654005c797b84b072642fd6b642ebba0d79028e71bdb2acbb3962/65333a7e/qo_oUU0UNiEpD-z6-zCIgdRB33v8cVyWATClBo2FTm36Wiufn6dFhyr1IoTkBcuIDOqKvX5GKvyJZKr2SKucCw%3D%3D?uid=0&filename=m5-forecating-accuracy.zip&disposition=attachment&hash=J5ovyMT7FWTmxkOjFLIMj3wuXKK82PqrkEXM2lsA3isVM3La/mWxkoo43Q/uGJZ5q/J6bpmRyOJonT3VoXnDag%3D%3D%3A/m5-forecating-accuracy.zip&limit=0&content_type=application%2Fzip&owner_uid=1199758960&fsize=48326531&hid=d3329dd040d2a0b0116f34e36f7142dc&media_type=compressed&tknv=v2'

Скачайте, данные по ссылке, которую вы получите после запуска предыдущей ячейки.

Для скачивания необходимо ввести команду: wget -O m5-forecating-accuracy.zip \<YOUR_DATA_LINK\>

P.S.: Ссылка с github может быть уже недействительной, поэтому нужно обязательно запустить у себя ячейку выше

In [28]:
# Вставьте здесь свой data_link
! wget -O m5-forecasting-accuracy.zip 'https://downloader.disk.yandex.ru/disk/2bfe1796ad8fef44ceb25f1b8332c7d99d05c5b1561b238bf21dedf38bc9edd5/65333993/qo_oUU0UNiEpD-z6-zCIgdRB33v8cVyWATClBo2FTm36Wiufn6dFhyr1IoTkBcuIDOqKvX5GKvyJZKr2SKucCw%3D%3D?uid=0&filename=m5-forecating-accuracy.zip&disposition=attachment&hash=J5ovyMT7FWTmxkOjFLIMj3wuXKK82PqrkEXM2lsA3isVM3La/mWxkoo43Q/uGJZ5q/J6bpmRyOJonT3VoXnDag%3D%3D%3A/m5-forecating-accuracy.zip&limit=0&content_type=application%2Fzip&owner_uid=1199758960&fsize=48326531&hid=d3329dd040d2a0b0116f34e36f7142dc&media_type=compressed&tknv=v2'

--2023-10-20 22:41:47--  https://downloader.disk.yandex.ru/disk/2bfe1796ad8fef44ceb25f1b8332c7d99d05c5b1561b238bf21dedf38bc9edd5/65333993/qo_oUU0UNiEpD-z6-zCIgdRB33v8cVyWATClBo2FTm36Wiufn6dFhyr1IoTkBcuIDOqKvX5GKvyJZKr2SKucCw%3D%3D?uid=0&filename=m5-forecating-accuracy.zip&disposition=attachment&hash=J5ovyMT7FWTmxkOjFLIMj3wuXKK82PqrkEXM2lsA3isVM3La/mWxkoo43Q/uGJZ5q/J6bpmRyOJonT3VoXnDag%3D%3D%3A/m5-forecating-accuracy.zip&limit=0&content_type=application%2Fzip&owner_uid=1199758960&fsize=48326531&hid=d3329dd040d2a0b0116f34e36f7142dc&media_type=compressed&tknv=v2
Resolving downloader.disk.yandex.ru (downloader.disk.yandex.ru)... 77.88.21.127, 2a02:6b8::2:127
Connecting to downloader.disk.yandex.ru (downloader.disk.yandex.ru)|77.88.21.127|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s235vla.storage.yandex.net/rdisk/2bfe1796ad8fef44ceb25f1b8332c7d99d05c5b1561b238bf21dedf38bc9edd5/65333993/qo_oUU0UNiEpD-z6-zCIgdRB33v8cVyWATClBo2FTm36Wiufn6dFhyr1IoTkBc

В этом проекте нужно работать с данными для предсказания спроса: [M5 Forecasting](https://www.kaggle.com/competitions/m5-forecasting-accuracy/data).

In [30]:
path = './m5-forecasting-accuracy'

In [31]:
import zipfile
with zipfile.ZipFile('./m5-forecasting-accuracy.zip', 'r') as zip_ref:
    zip_ref.extractall('./m5-forecasting-accuracy')

In [32]:
%ls $path

calendar.csv  sales_train_evaluation.csv  sample_submission.csv
__MACOSX/     sales_train_validation.csv  sell_prices.csv


In [33]:
# Зададим пути к файлам из датасета
file_calendar = f"{path}/calendar.csv"
file_validation = f"{path}/sales_train_validation.csv"
file_evaluation = f"{path}/sales_train_evaluation.csv"
file_prices = f"{path}/sell_prices.csv"

# Формат данных — CSV
file_type = "csv"
# Зададим параметры, как интерпретировать загруженные данные
# Определять типы колонок автоматически
infer_schema = "true"
# Интерпретируем первую строку в файле, как названия колонок
first_row_is_header = "true"
# Задаём разделитель между значениями колонок
delimiter = ","

# загружаем в Spark данные валидации
df_validation = (
    spark.read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_validation)
    # Также, можно указывать пути в hdfs или базы данных, например, Hive
#       .load('hdfs:///path_to_data/...')
)

# загружаем данные для теста
df_evaluation = (
    spark.read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_evaluation)
)
# загружаем данные с ценами
df_prices = (
    spark.read.format(file_type)
      .option("inferSchema", infer_schema)
      .option("header", first_row_is_header)
      .option("sep", delimiter)
      .load(file_prices)
)

# Возьмём первые 10 строк pyspark.sql.dataframe.DataFrame
# И выполним action-преобразование для преобразования в pandas.DataFrame
df_validation.limit(10).toPandas()

23/10/20 22:46:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,0,0,1,37,3,4,6,3,2,1
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,0,0,1,1,6,0,0,0,0,0
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,1,0,0,0,0,0,0,2,0,2


### `Spark DataFrame API`

* [Quickstart](https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html)
* [Документация](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html)

In [34]:
emp_data = [
    (1, 'Smith', 10),
    (2, 'Rose', 20),
    (3, 'Williams', 10),
    (4, 'Jones', 30),
    (5, 'Jones', None),
]
emp_columns = ['emp_id', 'name', 'dept_id']

emp_df = spark.createDataFrame(emp_data, emp_columns)
emp_df

DataFrame[emp_id: bigint, name: string, dept_id: bigint]

In [35]:
# выведем тип датафрейма
type(emp_df)

pyspark.sql.dataframe.DataFrame

Попробуем посмотреть, что находится в emp_df

In [37]:
emp_df

DataFrame[emp_id: bigint, name: string, dept_id: bigint]

Вывод DataFrame не показывает его содержимое, так как оно ещё не было вычислено, вычисления в Spark происходят только в момент вызова функций типа action.

Примеры action:
* `count()` — подсчитывает число строк в DataFrame
* `toPandas()` — преобразует Spark DataFrame в pandas DataFrame
* `collect()` — выполняет вычисление текущего Spark DataFrame и возвращает результат
* `show()` — `collect()` + pretty print результата

In [38]:
emp_df.show()

+------+--------+-------+
|emp_id|    name|dept_id|
+------+--------+-------+
|     1|   Smith|     10|
|     2|    Rose|     20|
|     3|Williams|     10|
|     4|   Jones|     30|
|     5|   Jones|   NULL|
+------+--------+-------+



Получим базовую информация о данных — названия колонок и тип данных, хранящийся в них

In [39]:
# DataFrame.columns - показывает названия колонок внутри spark-dataframe
print(emp_df.columns)
# DataFrame.schema - показывает их типы
emp_df.schema

['emp_id', 'name', 'dept_id']


StructType([StructField('emp_id', LongType(), True), StructField('name', StringType(), True), StructField('dept_id', LongType(), True)])

Многие методы дублируются по аналогии с `pandas.DataFrame`

In [40]:
# Уберём строки, содержащие хотя бы одно NULL значение
emp_df.dropna().show()

+------+--------+-------+
|emp_id|    name|dept_id|
+------+--------+-------+
|     1|   Smith|     10|
|     2|    Rose|     20|
|     3|Williams|     10|
|     4|   Jones|     30|
+------+--------+-------+



DataFrame состоит из колонок. Получение колонки возможно через атрибуты (точечная нотация) или через индексацию:

In [43]:
emp_df.name, emp_df['name']

(Column<'name'>, Column<'name'>)

Вместо самих значений колонок, в соответствии с принципом "ленивых" вычислений, возвращаются ссылки на них. Такие колонки могут участвовать в символьных вычислениях. Например, к ним можно применять арифметические, булевы операции.

In [44]:
# cозданим новую колонку с данными, на основе данных из колонок dep_id и emp_id,
# в соответствии с операцийми ниже
column_expr = (emp_df.dept_id - 20) / 10 > emp_df.emp_id
column_expr

Column<'(((dept_id - 20) / 10) > emp_id)'>

Так как не было выполненно ни одной операции тип action c новой колонкой, при выводе выдаётся только её символическая запись. Чтобы исправить это, запустим операцию типа-action show(), которая делает вывод посчитанного результата.

Полученные **колоночные выражения** (**column expression**) можно вычислять:

In [45]:
emp_df.select(column_expr).show()

+--------------------------------+
|(((dept_id - 20) / 10) > emp_id)|
+--------------------------------+
|                           false|
|                           false|
|                           false|
|                           false|
|                            NULL|
+--------------------------------+



Колонку можно переименовать, используя метод .alias(\<new_name\>) для данной колонки

In [46]:
emp_df.select((emp_df.dept_id ** 2).alias('dept_id squared')).show()

+---------------+
|dept_id squared|
+---------------+
|          100.0|
|          400.0|
|          100.0|
|          900.0|
|           NULL|
+---------------+



Для DataFrame доступны SQL подобные операции, например, `join`:

In [48]:
dept_data = [
    ('Finance', 10),
    ('Marketing', 20),
    ('Sales', 30),
    ('IT', 40),
]
dept_columns = ['dept_name', 'dept_id']

# создаём spark-датафрейм
dept_df = spark.createDataFrame(dept_data, dept_columns)
dept_df.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



In [50]:
# через метод .join() объеденим emp_df с dept_df по ключю dept_id
emp_df.join(dept_df, how='inner', on=['dept_id']).show()

+-------+------+--------+---------+
|dept_id|emp_id|    name|dept_name|
+-------+------+--------+---------+
|     10|     1|   Smith|  Finance|
|     10|     3|Williams|  Finance|
|     20|     2|    Rose|Marketing|
|     30|     4|   Jones|    Sales|
+-------+------+--------+---------+



В предыдущем примере объединение таблиц происходило только по пересекающимся (параметр how='inner') значениям dept_id, теперь объденим таблицы по всевозможным ключам (параметр how='outer').

In [53]:
emp_df.join(dept_df, how='outer', on=['dept_id']).show()

+-------+------+--------+---------+
|dept_id|emp_id|    name|dept_name|
+-------+------+--------+---------+
|   NULL|     5|   Jones|     NULL|
|     10|     1|   Smith|  Finance|
|     10|     3|Williams|  Finance|
|     20|     2|    Rose|Marketing|
|     30|     4|   Jones|    Sales|
|     40|  NULL|    NULL|       IT|
+-------+------+--------+---------+



Также, доступна фильтрация и сортировка:

In [54]:
# метод where отвечает за фильрации, внутри которого указываются условия фильтрации
# метод sort производит сортировку по полю/полям, которые указаны в его аргументах
(
    emp_df
      .join(dept_df, how='outer', on=['dept_id'])
      # Обратите внимание на колоночное выражение в фильтре
      .where((emp_df['name'] == 'Smith') | (emp_df['name'] == 'Rose'))
      .sort('dept_id')
      .show()
)

+-------+------+-----+---------+
|dept_id|emp_id| name|dept_name|
+-------+------+-----+---------+
|     10|     1|Smith|  Finance|
|     20|     2| Rose|Marketing|
+-------+------+-----+---------+



Работа с колонками обычно выполняется через колоночные выражения. Их можно использовать, например, для выполнения join:

In [55]:
emp_columns_renamed = ['emp_id', 'name', 'emp_dept_id']

emp_renamed_df = spark.createDataFrame(emp_data, emp_columns_renamed)
emp_renamed_df.show()

+------+--------+-----------+
|emp_id|    name|emp_dept_id|
+------+--------+-----------+
|     1|   Smith|         10|
|     2|    Rose|         20|
|     3|Williams|         10|
|     4|   Jones|         30|
|     5|   Jones|       NULL|
+------+--------+-----------+



In [56]:
# теперь вместо параметра  on=['dept_id'] в методе join используем колоночное выражение
emp_renamed_df.join(
    dept_df, emp_renamed_df.emp_dept_id == dept_df.dept_id,  how='inner'
).show()

+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     1|   Smith|         10|  Finance|     10|
|     3|Williams|         10|  Finance|     10|
|     2|    Rose|         20|Marketing|     20|
|     4|   Jones|         30|    Sales|     30|
+------+--------+-----------+---------+-------+



Переименование колонок также возможно:

In [57]:
# .withColumnRenamed(<old_name>, <new_name>)
(
    emp_renamed_df
      .withColumnRenamed('emp_dept_id', 'dept_id')
      .join(
          dept_df, 'dept_id',  how='inner'
      )
      .show()
)

+-------+------+--------+---------+
|dept_id|emp_id|    name|dept_name|
+-------+------+--------+---------+
|     10|     1|   Smith|  Finance|
|     10|     3|Williams|  Finance|
|     20|     2|    Rose|Marketing|
|     30|     4|   Jones|    Sales|
+-------+------+--------+---------+



Для преобразования колонок в модуле `pyspark.sql.functions` содержится большой набор вспомогательных функций. Например:
* Вспомогательные: `lit`, `col`, ...
* Поэлементные математические функции: `cos`, `sin`, `round`, ...
* Поэлементные функции для работы датами и временем: `dayofmonth`, ...
* Агрегаторы: `sum`, `mean`, ...
* Функции для работы с коллекциями (сложными данными, хранящимся в колонке): `array_sort`, `concat`, ...
* Сортировки: `asc`, ...
* Строковые функции: `concat_ws`, `lower`, `split`, ...
* Оконные функции: `lag`, ...
* Преобразования с пользовательскими функциями: `udf_pandas`, ...

**Всегда перед написанием кода нужно подумать, нет ли уже готовой функции в данном модуле. Использование готовых функции существенно влияет на скорость вычислений.**

In [58]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType

Часто для применения функций нужно поменять тип колонки, за что отвечает метод .cast(\<NEW_COLUMN_TYPE\>)

In [60]:
emp_with_date = (
    emp_df
        .dropna()
        .withColumn(
            'hire_date', 
            # Конструируем дату в формате yyyy-mm-dd 
            F.concat_ws(
                '-',
                # Придумываем год
                (1990 + emp_df.dept_id).cast(StringType()),
                # Придумываем месяц
                F.concat(F.lit('0'), emp_df.emp_id.cast(StringType())), 
                # Придумываем день
                emp_df.dept_id.cast(StringType())
            ).cast(DateType())
        )
)
emp_with_date.show()

+------+--------+-------+----------+
|emp_id|    name|dept_id| hire_date|
+------+--------+-------+----------+
|     1|   Smith|     10|2000-01-10|
|     2|    Rose|     20|2010-02-20|
|     3|Williams|     10|2000-03-10|
|     4|   Jones|     30|2020-04-30|
+------+--------+-------+----------+



In [63]:
emp_with_date.select(
    # вызваем arccos для колонки
    F.acos(emp_with_date.emp_id / 4),
    # берём от колонки с датой только значение года
    F.year(emp_with_date.hire_date), 
    # заменяем все вхождения smith на смит и переименовываем её в processed_name
    F.regexp_replace(F.lower(emp_with_date.name), 'smith', 'cмит').alias('processed_name') 
).show()

+------------------+---------------+--------------+
|ACOS((emp_id / 4))|year(hire_date)|processed_name|
+------------------+---------------+--------------+
| 1.318116071652818|           2000|          cмит|
|1.0471975511965979|           2010|          rose|
|0.7227342478134157|           2000|      williams|
|               0.0|           2020|         jones|
+------------------+---------------+--------------+



### `Spark SQL API`

Работа с таблицами в Spark возможна не только в Pandas-подобном интерфейсе, также поддерживается SQL выражения для обработки.

In [64]:
add_data = [
    (1, '1523 Main St', 'SFO', 'CA'),
    (2, '3453 Orange St', 'SFO', 'NY'),
    (3, '34 Warner St', 'Jersey', 'NJ'),
    (4, '221 Cavalier St', 'Newark', 'DE'),
    (5, '789 Walnut St', 'Sandiago', 'CA')
]
add_columns = ['emp_id', 'address', 'city', 'state']
# создадим тестовую таблицу
add_df = spark.createDataFrame(add_data, add_columns)
add_df.show()

+------+---------------+--------+-----+
|emp_id|        address|    city|state|
+------+---------------+--------+-----+
|     1|   1523 Main St|     SFO|   CA|
|     2| 3453 Orange St|     SFO|   NY|
|     3|   34 Warner St|  Jersey|   NJ|
|     4|221 Cavalier St|  Newark|   DE|
|     5|  789 Walnut St|Sandiago|   CA|
+------+---------------+--------+-----+



Spark позволяет использовать DataFrame в качестве таблиц в регулярных SQL запросах:

In [69]:
# создаём таблицу с именем EMP на основе датафрейма emp_df
emp_df.createOrReplaceTempView('EMP')
# создаём таблицу с именем DEPT на основе датафрейма dept_df
dept_df.createOrReplaceTempView('DEPT')
# создаём таблицу с именем DEPT на основе датафрейма add_df
add_df.createOrReplaceTempView('ADD')

In [70]:
# выполняем сам SQL запрос над данными таблицами
spark.sql('''
    select * from EMP e, DEPT d, ADD a
    where e.dept_id == d.dept_id and e.emp_id == a.emp_id
''').show()

+------+--------+-------+---------+-------+------+---------------+------+-----+
|emp_id|    name|dept_id|dept_name|dept_id|emp_id|        address|  city|state|
+------+--------+-------+---------+-------+------+---------------+------+-----+
|     1|   Smith|     10|  Finance|     10|     1|   1523 Main St|   SFO|   CA|
|     2|    Rose|     20|Marketing|     20|     2| 3453 Orange St|   SFO|   NY|
|     3|Williams|     10|  Finance|     10|     3|   34 Warner St|Jersey|   NJ|
|     4|   Jones|     30|    Sales|     30|     4|221 Cavalier St|Newark|   DE|
+------+--------+-------+---------+-------+------+---------------+------+-----+



### `Ещё базовые операции над Spark DataFrame`

In [71]:
data = [
    ('James', 'Sales', 3000),
    ('Michael', 'Sales', 4600),
    ('Robert', 'Sales', 4100),
    ('Maria', 'Finance', 3000),
    ('James', 'Sales', 3000),
    ('Scott', 'Finance', 3300),
    ('Jen', 'Finance', 3900),
    ('Jeff', ' Marketing', 3000),
    ('Kumar', 'Marketing', 2000),
    ('Saif', 'Sales', 4100),
]
columns = ['Name', 'Dept', 'Salary']

df = spark.createDataFrame(data, columns)
df.show()

+-------+----------+------+
|   Name|      Dept|Salary|
+-------+----------+------+
|  James|     Sales|  3000|
|Michael|     Sales|  4600|
| Robert|     Sales|  4100|
|  Maria|   Finance|  3000|
|  James|     Sales|  3000|
|  Scott|   Finance|  3300|
|    Jen|   Finance|  3900|
|   Jeff| Marketing|  3000|
|  Kumar| Marketing|  2000|
|   Saif|     Sales|  4100|
+-------+----------+------+



In [72]:
# метод .distinct отбирает только уникальные строки в датафрейме
df.distinct().show()

+-------+----------+------+
|   Name|      Dept|Salary|
+-------+----------+------+
|Michael|     Sales|  4600|
|  James|     Sales|  3000|
| Robert|     Sales|  4100|
|  Maria|   Finance|  3000|
|    Jen|   Finance|  3900|
|  Scott|   Finance|  3300|
|  Kumar| Marketing|  2000|
|   Saif|     Sales|  4100|
|   Jeff| Marketing|  3000|
+-------+----------+------+



In [73]:
# метод count() считаем число строк в датафрейме
# так как count относится к action операциям, то метод show можно не вызывать
df.distinct().count()

9

Также, возможно использовать группировку и агрегаты:

In [34]:
# сгрупировать данные по департаменту и посчитать суммарную зарплату всех сотрудникаов в нём
df.groupBy('Dept').sum().collect()

[Row(Dept='Sales', sum(Salary)=18800),
 Row(Dept='Finance', sum(Salary)=10200),
 Row(Dept=' Marketing', sum(Salary)=3000),
 Row(Dept='Marketing', sum(Salary)=2000)]

### `IO операции`

После того как было произведены все операции с данными нужно сохранить результаты их работы на диск

In [74]:
base_statistics = df.select(
    F.min('Salary').alias('min_salary'),
    F.mean('Salary').alias('mean_salary'),
    F.max('Salary').alias('max_salary')
)
# Пока никаких вычислений не произошло
base_statistics

DataFrame[min_salary: bigint, mean_salary: double, max_salary: bigint]

In [75]:
# записываем данные на диск в CSV формате формате
base_statistics.write.csv('./base_statistics.csv', header=True)
# записываем данные на диск в специальном spark-формате parquet
base_statistics.write.parquet('./base_statistics.parquet')

In [76]:
%ls ./base_statistics.csv/
%pycat ./base_statistics.csv/part-00000-5f1b82c2-0fe0-4305-8c71-067297fd29b2-c000.csv

part-00000-cb924d66-c4d7-4a7d-9c4a-9991b308fa9a-c000.csv  _SUCCESS
Error: no such file, variable, URL, history range or macro


Для просмотра parquet-файлов нужны специльные инструменты

In [80]:
%ls ./base_statistics.parquet/
! parquet-tools inspect ./base_statistics.parquet/part-00000-17fd217a-ca74-4556-bb0b-991cee8924dc-c000.snappy.parquet

part-00000-17fd217a-ca74-4556-bb0b-991cee8924dc-c000.snappy.parquet  _SUCCESS

############ file meta data ############
created_by: parquet-mr version 1.13.1 (build db4183109d5b734ec5930d870cdae161e408ddba)
num_columns: 3
num_rows: 1
num_row_groups: 1
format_version: 1.0
serialized_size: 789


############ Columns ############
min_salary
mean_salary
max_salary

############ Column(min_salary) ############
name: min_salary
path: min_salary
max_definition_level: 1
max_repetition_level: 0
physical_type: INT64
logical_type: None
converted_type (legacy): NONE
compression: SNAPPY (space_saved: -5%)

############ Column(mean_salary) ############
name: mean_salary
path: mean_salary
max_definition_level: 1
max_repetition_level: 0
physical_type: DOUBLE
logical_type: None
converted_type (legacy): NONE
compression: SNAPPY (space_saved: -5%)

############ Column(max_salary) ############
name: max_salary
path: max_salary
max_definition_level: 1
max_repetition_level: 0
physical_type: INT64
logical_ty

Теперь произвём операцию чтения сохранённых данных

In [39]:
loaded_df = (
    spark.read
        .format('csv')
        .option("inferSchema", True)
        .option("header", True)
        .option("sep", ',')
        .load('./base_statistics.csv')
)
loaded_df, loaded_df.show()

+----------+-----------+----------+
|min_salary|mean_salary|max_salary|
+----------+-----------+----------+
|      2000|     3400.0|      4600|
+----------+-----------+----------+



(DataFrame[min_salary: int, mean_salary: double, max_salary: int], None)

Как видно из вывода все данные были сохранены корректно

##### `Завершение работы`

После завершения работы не забываем осовободить ресурсы и остановить текущую сессию spark

In [81]:
sc.stop()